In [26]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from tqdm import tqdm
import lightgbm as lgbm
import re
from sklearn.ensemble import RandomForestRegressor 

import os
import json
%matplotlib inline

import keras
import tensorflow as tf

from keras.layers import Dense, Activation, Conv2D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D
import keras.backend as K
from keras.regularizers import l1_l2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, Callback, EarlyStopping
from sklearn.datasets.samples_generator import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential, load_model

In [27]:
print("keras_version : " + str(keras.__version__))
print("numpy_version : " + str(np.__version__))
print("pandas_version : " + str(pd.__version__))
print("tensorflow_version :" + str(tf.__version__))

keras_version : 2.4.3
numpy_version : 1.18.5
pandas_version : 1.1.4
tensorflow_version :2.3.0


In [28]:
train = pd.read_csv("/content/drive/My Drive/Dacon_튜터/train_features.csv", index_col=0)
test = pd.read_csv("/content/drive/My Drive/Dacon_튜터/test_features.csv", index_col=0)
y_train = pd.read_csv("/content/drive/My Drive/Dacon_튜터/train_target.csv", index_col=0)
y_test = pd.read_csv("/content/drive/My Drive/Dacon_튜터/sample_submission.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [29]:
def transform1(train, n1, n2):
    train_time = train.copy()
    train_time['S1'] = [1 if i != 0 else 0 for i in train['S1']]
    train_time['S2'] = [1 if i != 0 else 0 for i in train['S2']]
    train_time['S3'] = [1 if i != 0 else 0 for i in train['S3']]
    train_time['S4'] = [1 if i != 0 else 0 for i in train['S4']]
    train_time.drop(['Time'], axis=1, inplace=True)
    
    train_time = train_time.sum(axis=1)
    train_time = train_time.transform(lambda x: (x-x.mean())/(x.std()))
    train_time = np.array(train_time).reshape(n1, n2, 1, 1)
    
    return train_time

In [30]:
train = train.transform(lambda x: (x-x.mean())/(x.std()))
test = test.transform(lambda x: (x-x.mean())/(x.std()))

###

X_data = train.iloc[:,:]
X_data = np.array(X_data).reshape((2800,375,5,1))

X_data_test = test.iloc[:,:]
X_data_test = np.array(X_data_test).reshape((700,375,5,1))

Y_data = np.array(y_train.copy())

In [57]:
X_data_Case4 = X_data.copy()
X_data_test_Case4 = X_data_test.copy()
Y_data_Case4 = Y_data.copy()

모델링

In [32]:
import math
from keras.callbacks import Callback
class CosineAnnealingScheduler(Callback):
    """Cosine annealing scheduler.
    """

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        #if self.verbose &gt; 0:
        #    print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
        #          'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [33]:
import numpy as np

def kaeri_metric(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: KAERI metric
    '''
    
    return 0.5 * E1(y_true, y_pred) + 0.5 * E2(y_true, y_pred)


### E1과 E2는 아래에 정의됨 ###

def E1(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: distance error normalized with 2e+04
    '''
    
    _t, _p = np.array(y_true)[:,:2], np.array(y_pred)[:,:2]
    
    return np.mean(np.sum(np.square(_t - _p), axis = 1) / 2e+04)


def E2(y_true, y_pred):
    '''
    y_true: dataframe with true values of X,Y,M,V
    y_pred: dataframe with pred values of X,Y,M,V
    
    return: sum of mass and velocity's mean squared percentage error
    '''
    
    _t, _p = np.array(y_true)[:,2:], np.array(y_pred)[:,2:]
    
    
    return np.mean(np.sum(np.square((_t - _p) / (_t + 1e-06)), axis = 1))

In [34]:
weight1 = np.array([1,1,0,0])
weight2 = np.array([0,0,1,1])

def my_loss(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult))


def my_loss_E1(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred)*weight1)/2e+04

def my_loss_E2(y_true, y_pred):
    divResult = Lambda(lambda x: x[0]/x[1])([(y_pred-y_true),(y_true+0.000001)])
    return K.mean(K.square(divResult)*weight2)

In [35]:
def plot_error(type_id,pred,true):
    print(pred.shape)

    if type_id == 0:
        _name = 'x_pos'
    elif type_id == 1:
        _name = 'y_pos'
    elif type_id == 2:
        _name = 'mass'
    elif type_id == 3:
        _name = 'velocity'
    elif type_id == 4:
        _name = "distance"
    else:
        _name = 'error'

    x_coord = np.arange(1,pred.shape[0]+1,1)
    if type_id < 2:
        Err_m = (pred[:,type_id] - true[:,type_id])
    elif type_id < 4:
        Err_m = ((pred[:,type_id] - true[:,type_id])/true[:,type_id])*100
    else:
        Err_m = ((pred[:,0]-true[:,0])**2+(pred[:,1]-true[:,1])**2)**0.5


    fig = plt.figure(figsize=(8,6))
    # plt.rcParams["font.family"]="Times New Roman"
    plt.rcParams["font.size"]=15
    plt.scatter(x_coord, Err_m, marker='o')
    plt.title("%s Prediction for Training Data" % _name, size=20)
    plt.xlabel("Data ID", labelpad=10, size=20)
    plt.ylabel("Prediction Error of %s," % _name, labelpad=10, size=20)
    plt.xticks(size=15)
    plt.yticks(size=15)
    plt.ylim(-100., 100.)
    plt.xlim(0, pred.shape[0]+1)

    plt.show()
    
    print(np.std(Err_m))
    print(np.max(Err_m))
    print(np.min(Err_m))
    return Err_m

In [36]:
def set_model(train_target, n_sam,n_col, regularizer_True):  # 0:x,y, 1:m, 2:v
    
    activation = 'elu'
    padding = 'same'
    model = Sequential()
    nf = 16
    fs = (3,1)
    
    model.add(Conv2D(nf,fs, padding=padding, activation=activation,input_shape=(n_sam,n_col,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*2,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*4,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*8,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*16,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))

    model.add(Conv2D(nf*32,fs, padding=padding, activation=activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    model.add(Flatten())
    model.add(Dense(128, activation ='elu'))
    model.add(Dense(64, activation ='elu'))
    model.add(Dense(32, activation ='elu'))
    model.add(Dense(16, activation ='elu'))
    
    if regularizer_True:
        model.add(Dense(4, kernel_regularizer=l1_l2(l1=0.001)))
    else:
        model.add(Dense(4))
    

    optimizer = keras.optimizers.Adam()
    #optimizer = keras.optimizers.RMSprop()
    
    global weight2
    if train_target == 1: # only for M
        weight2 = np.array([0,0,1,0])
    else: # only for V
        weight2 = np.array([0,0,0,1])
       
    if train_target==0:
        model.compile(loss=my_loss_E1,
                  optimizer=optimizer,
                 )
    else:
        model.compile(loss=my_loss_E2,
                  optimizer=optimizer,
                 )
       
    model.summary()

    return model

In [65]:
def train_CNN(model,X,Y,X_val,Y_val, save_point, enum, train_target, scheduler_True):
    
    from numpy.random import seed
    seed(777)
    
    MODEL_SAVE_FOLDER_PATH = './model/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '{epoch:02d}-{val_loss:.4f}.hdf5'
    best_save = ModelCheckpoint('best_m_' + str(save_point) + "_" + str(enum) + "_" + str(train_target) + '.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    cosine_scheduler = CosineAnnealingScheduler(T_max=100, eta_max=6e-3, eta_min=1e-6)

    early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
    
    my_seed = 777
    np.random.seed(my_seed)
    if scheduler_True:
        history = model.fit(X, Y,
                      epochs=10000,
                      batch_size=128,
                      shuffle=True,
                      validation_data=(X_val, Y_val),
                      verbose = 2,
                      callbacks=[best_save, early_stop,cosine_scheduler],
                      )
    else:
        history = model.fit(X, Y,
                      epochs=10000,
                      batch_size=128,
                      shuffle=True,
                      validation_data=(X_val, Y_val),
                      verbose = 2,
                      callbacks=[best_save, early_stop],
                      )
    

    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(history.history['loss'], 'y', label='train loss')
    loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')
    plt.show()    
    
    return model

In [38]:
def load_best_model(train_target, save_point, enum):

    if train_target == 0:
        model = load_model('best_m_' + str(save_point) + "_" + str(enum) + "_" + str(train_target) + '.hdf5' , custom_objects={'my_loss_E1': my_loss, })
    else:
        model = load_model('best_m_' + str(save_point) + "_" + str(enum) + "_" + str(train_target) + '.hdf5' , custom_objects={'my_loss_E2': my_loss, })

    score = model.evaluate(X_data, Y_data, verbose=0)
    print('loss:', score)

    pred = model.predict(X_data)

    i=0

    print('정답(original):', Y_data[i])
    print('예측값(original):', pred[i])

    print(E1(pred, Y_data))
    print(E2(pred, Y_data))
    #print(E2M(pred, Y_data))
    #print(E2V(pred, Y_data))    
    
    if train_target ==0:
        plot_error(4,pred,Y_data)
    elif train_target ==1:
        plot_error(2,pred,Y_data)
    elif train_target ==2:
        plot_error(3,pred,Y_data)    
    
    return model

In [59]:
lr = 3e-4
lr_d = 0.0
patience = 30
dr_rate = 0

def run_model(X_data, X_data_test, Y_data, save_point,scheduler_True,regularizer_True):
    submit = pd.read_csv("/content/drive/My Drive/Dacon_튜터/sample_submission.csv", index_col=0)
    submit.iloc[:,:] = 0

    lr = 3e-4
    lr_d = 0.0
    patience = 30
    dr_rate = 0
    
    n_col = X_data.shape[2]
    n_sam = X_data.shape[1]
    
    for enum, (train_index,valid_index) in enumerate(kf.split(X_data)):
        X_train = X_data[train_index]
        Y_train = Y_data[train_index]
    
        X_val = X_data[valid_index]
        Y_val = Y_data[valid_index]
    
    
        for train_target in range(3):
            model = set_model(train_target, n_sam,n_col, regularizer_True)
        
            train_CNN(model,X_train, Y_train, X_val, Y_val, save_point, enum, train_target,scheduler_True)    
            best_model = load_best_model(train_target, save_point, enum)

   
            pred_data_test = best_model.predict(X_data_test)
            pred_data_valid = best_model.predict(X_val)
    
            if train_target == 0: # x,y 학습
                submit.iloc[:,0] += pred_data_test[:,0]/5
                submit.iloc[:,1] += pred_data_test[:,1]/5
                
            elif train_target == 1: # m 학습
                submit.iloc[:,2] += pred_data_test[:,2]/5
        
            elif train_target == 2: # v 학습
                submit.iloc[:,3] += pred_data_test[:,3]/5    
    
    return submit

In [60]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [61]:
X_data.shape

(2800, 375, 5, 1)

In [62]:
print("Case4 nrow : " + str(X_data_Case4.shape[1]) + " / Case4 ncol : " + str(X_data_Case4.shape[2]))
print("Case4 nrow : " + str(X_data_test_Case4.shape[1]) + " / Case4 ncol : " + str(X_data_test_Case4.shape[2]))

Case4 nrow : 375 / Case4 ncol : 5
Case4 nrow : 375 / Case4 ncol : 5


In [63]:
X_data_Case4.shape

(2800, 375, 5, 1)

In [67]:
submit = run_model(X_data,X_data_test, Y_data, 1,True,False)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 375, 5, 16)        64        
_________________________________________________________________
batch_normalization_36 (Batc (None, 375, 5, 16)        64        
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 187, 5, 16)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 187, 5, 32)        1568      
_________________________________________________________________
batch_normalization_37 (Batc (None, 187, 5, 32)        128       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 93, 5, 32)         0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 93, 5, 64)        

KeyboardInterrupt: ignored